In [1]:
import pyedflib
import numpy as np
import pandas as pd

In [2]:
import importlib

In [3]:
import scipy.signal as signal

from scipy.signal import hilbert

In [4]:
import arrow

In [5]:
from datetime import datetime as dt
from datetime import timedelta as td

In [6]:
from bs4 import BeautifulSoup

In [7]:
import matplotlib.pyplot as plt

In [55]:
import taa_calc
importlib.reload(taa_calc)
from taa_calc import (
    get_edf_signal_indices,
    read_edf_signals,
    read_edf_signals_file,
    resample_signals,
    preprocess,
    fund_freq,
    avg_resp_period,
    calc_windows,
    calc_taa,
    read_sleep_stages,
    read_resp_events,
    chart_taa,
    compute_perc_taa,
    read_files,
)

In [9]:
from pathlib import Path

In [28]:
base_path = Path("./local-data/chat/polysomnography/")
base_edf_path = base_path / Path("edfs/")
followup_path = Path("./local-data/chat/polysomnography/edfs/followup/")
baseline_path = Path("./local-data/chat/polysomnography/edfs/baseline/")

In [ ]:
poly_raw_data = pd.read_csv('local-data/chat/files/datasets/chat-baseline-dataset-0.10.0.csv',
                        header=0)
poly_data = poly_raw_data[['nsrrid', 'arm']].dropna()
poly_data['arm'] = poly_data['arm'].astype(np.int)
poly_data['treatment'] = np.where(poly_data['arm'] == 1, 'surgery', 'wait')

In [40]:
import re
edf_id_re = re.compile(r'chat-(baseline|followup)-(\d+)\.edf')
edf_id_files = []
edf_filenames = []
def edf_base_file(edf_filename):
    fname = str(edf_filename.relative_to(base_edf_path))
    return fname[:-4]
for followup_edf_file in sorted(followup_path.glob("*.edf")):
    filename = followup_edf_file.name
    edf_id = edf_id_re.match(filename)[2]
    baseline_edf_fname = baseline_path / f"chat-baseline-{edf_id}.edf"
    #print(baseline_edf_fname)
    if baseline_edf_fname.is_file():
        edf_filenames.append(edf_base_file(baseline_edf_fname))
        edf_filenames.append(edf_base_file(followup_edf_file))
        edf_id_files.append((edf_id, followup_edf_file, baseline_edf_fname))
    else:
        print("not in baseline")

In [47]:
edf_filenames[:10]

['baseline/chat-baseline-300001',
 'followup/chat-followup-300001',
 'baseline/chat-baseline-300002',
 'followup/chat-followup-300002',
 'baseline/chat-baseline-300004',
 'followup/chat-followup-300004',
 'baseline/chat-baseline-300007',
 'followup/chat-followup-300007',
 'baseline/chat-baseline-300013',
 'followup/chat-followup-300013']

In [78]:
res_df_8_12 = read_files(edf_filenames[8:12])

  0 processing baseline/chat-baseline-300013
baseline/chat-baseline-300013 sample frequency is 200.


/Users/jbranam/files/study/iums/d590_dsip/aisdb/aisdb/taa_calc.py:375: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 375 of the file /Users/jbranam/files/study/iums/d590_dsip/aisdb/aisdb/taa_calc.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  annot = BeautifulSoup(fp)


avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  1 processing followup/chat-followup-300013
followup/chat-followup-300013 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  2 processing baseline/chat-baseline-300014
baseline/chat-baseline-300014 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  3 processing followup/chat-followup-300014
followup/chat-followup-300014 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0


In [58]:
res_df_8

,comment,orig_freq,duration_sec,sleep_seconds,hyp_apnea_seconds,asleep_non_event_seconds,time_in_async_25p,time_in_async_50p,time_in_async_75p,time_in_async_25p_wevents,time_in_async_50p_wevents,time_in_async_75p_wevents,invalid_taa_periods
filename,,,,,,,,,,,,,
baseline/chat-baseline-300001,success,128,32258,28650,247,28411,0.0140,0.0018,0.0008,0.0146,0.0019,0.0008,347
followup/chat-followup-300001,success,128,32684,25620,112,25508,0.1865,0.1700,0.1520,0.1875,0.1703,0.1518,997
baseline/chat-baseline-300002,success,200,45349,29130,403,28727,0.0550,0.0079,0.0025,0.0547,0.0079,0.0026,439
followup/chat-followup-300002,success,200,36494,30600,611,30006,0.1603,0.0921,0.0670,0.1610,0.0910,0.0657,1269
baseline/chat-baseline-300004,success,200,37545,27750,512,27247,0.2615,0.1455,0.0988,0.2661,0.1498,0.1007,634
followup/chat-followup-300004,success,200,36420,27240,41,27199,0.0240,0.0051,0.0018,0.0241,0.0051,0.0019,934
baseline/chat-baseline-300007,success,32,35792,27750,169,27585,0.0351,0.0056,0.0032,0.0354,0.0054,0.0031,254
followup/chat-followup-300007,success,200,32780,23940,109,23831,0.0299,0.0037,0.0018,0.0299,0.0037,0.0018,419


In [85]:
res_df_8_12['nsrrid'] = res_df_8_12.index.str[-6:]

In [84]:
res_df_8['nsrrid'] = res_df_8.index.str[-6:]

In [113]:
res_df_12['study'] = res_df_12.filename.str.split('/').str[0]

In [86]:
res_df_12 = pd.concat([res_df_8, res_df_8_12])

In [114]:
res_df_12

,filename,comment,orig_freq,duration_sec,sleep_seconds,hyp_apnea_seconds,asleep_non_event_seconds,time_in_async_25p,time_in_async_50p,time_in_async_75p,time_in_async_25p_wevents,time_in_async_50p_wevents,time_in_async_75p_wevents,invalid_taa_periods,nsrrid,arm,treatment,study
0,baseline/chat-baseline-300001,success,128,32258,28650,247,28411,0.0140,0.0018,0.0008,0.0146,0.0019,0.0008,347,300001,2,wait,baseline
1,followup/chat-followup-300001,success,128,32684,25620,112,25508,0.1865,0.1700,0.1520,0.1875,0.1703,0.1518,997,300001,2,wait,followup
2,baseline/chat-baseline-300002,success,200,45349,29130,403,28727,0.0550,0.0079,0.0025,0.0547,0.0079,0.0026,439,300002,2,wait,baseline
3,followup/chat-followup-300002,success,200,36494,30600,611,30006,0.1603,0.0921,0.0670,0.1610,0.0910,0.0657,1269,300002,2,wait,followup
4,baseline/chat-baseline-300004,success,200,37545,27750,512,27247,0.2615,0.1455,0.0988,0.2661,0.1498,0.1007,634,300004,1,surgery,baseline
5,followup/chat-followup-300004,success,200,36420,27240,41,27199,0.0240,0.0051,0.0018,0.0241,0.0051,0.0019,934,300004,1,surgery,followup
6,baseline/chat-baseline-300007,success,32,35792,27750,169,27585,0.0351,0.0056,0.0032,0.0354,0.0054,0.0031,254,300007,1,surgery,baseline
7,followup/chat-followup-300007,success,200,32780,23940,109,23831,0.0299,0.0037,0.0018,0.0299,0.0037,0.0018,419,300007,1,surgery,followup
8,baseline/chat-baseline-300013,success,200,37446,30120,254,29866,0.1252,0.0057,0.0022,0.1247,0.0054,0.0023,520,300013,1,surgery,baseline
9,followup/chat-followup-300013,success,200,39396,29370,139,29231,0.0128,0.0043,0.0022,0.0127,0.0043,0.0022,354,300013,1,surgery,followup


In [100]:
poly_data['nsrrid'] = poly_data['nsrrid'].astype(str)
poly_data.head(7)

,nsrrid,arm,treatment
0,300001,2,wait
1,300002,2,wait
2,300004,1,surgery
3,300007,1,surgery
4,300008,1,surgery
5,300013,1,surgery
6,300014,2,wait


In [109]:
res_df_12 = res_df_12.reset_index().merge(poly_data, on='nsrrid')

In [110]:
res_df_12

,filename,comment,orig_freq,duration_sec,sleep_seconds,hyp_apnea_seconds,asleep_non_event_seconds,time_in_async_25p,time_in_async_50p,time_in_async_75p,time_in_async_25p_wevents,time_in_async_50p_wevents,time_in_async_75p_wevents,invalid_taa_periods,nsrrid,arm,treatment
0,baseline/chat-baseline-300001,success,128,32258,28650,247,28411,0.0140,0.0018,0.0008,0.0146,0.0019,0.0008,347,300001,2,wait
1,followup/chat-followup-300001,success,128,32684,25620,112,25508,0.1865,0.1700,0.1520,0.1875,0.1703,0.1518,997,300001,2,wait
2,baseline/chat-baseline-300002,success,200,45349,29130,403,28727,0.0550,0.0079,0.0025,0.0547,0.0079,0.0026,439,300002,2,wait
3,followup/chat-followup-300002,success,200,36494,30600,611,30006,0.1603,0.0921,0.0670,0.1610,0.0910,0.0657,1269,300002,2,wait
4,baseline/chat-baseline-300004,success,200,37545,27750,512,27247,0.2615,0.1455,0.0988,0.2661,0.1498,0.1007,634,300004,1,surgery
5,followup/chat-followup-300004,success,200,36420,27240,41,27199,0.0240,0.0051,0.0018,0.0241,0.0051,0.0019,934,300004,1,surgery
6,baseline/chat-baseline-300007,success,32,35792,27750,169,27585,0.0351,0.0056,0.0032,0.0354,0.0054,0.0031,254,300007,1,surgery
7,followup/chat-followup-300007,success,200,32780,23940,109,23831,0.0299,0.0037,0.0018,0.0299,0.0037,0.0018,419,300007,1,surgery
8,baseline/chat-baseline-300013,success,200,37446,30120,254,29866,0.1252,0.0057,0.0022,0.1247,0.0054,0.0023,520,300013,1,surgery
9,followup/chat-followup-300013,success,200,39396,29370,139,29231,0.0128,0.0043,0.0022,0.0127,0.0043,0.0022,354,300013,1,surgery
